In [1]:
# load and optionally install required packages
if (!require('pacman')) install.packages('pacman')
pacman::p_load(
    dplyr,
    readr,
#    reshape2,
    SnowballC,
    stopwords,
    stringr,
    textclean,
#    tidytext,
#    tidyverse,
    tm,
    textreg,
#    topicdoc,
#    topicmodels,
    qdap,
    qdapRegex
)

Loading required package: pacman



In [2]:
data <- read_csv('./data/generated/debotted_combined_distinct.csv')

Rows: 129101 Columns: 51
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (31): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
clean_and_tokenize <- function(text, language){
    payload <- text %>%
        str_replace('^RT', '') %>%
        str_to_lower() %>%
        replace_contraction() %>%
        #replace_emoji() # replacements have spaces
        #replace_emoticons()
        str_replace_all(pattern = '#([a-z0-9])', replacement = function(text){paste0('hashtag',gsub('^#', '', text),'hashtag')}) %>% # place 'hashtag' before and after to circumvent punctuation filter and stemming
        rm_url() %>%
        str_remove_all(pattern = '@\\w+') %>%
        str_remove_all(pattern = '\\$?\\d+\\w*') %>%
        str_replace_all(pattern = '[[:punct:]]', replacement = '') %>%
        str_squish() %>%
        str_trim() %>%
        stemDocument(language = language) %>%
        qdap::rm_stopwords(tm::stopwords(language)) %>%
        sapply(paste, collapse=" ") %>%
        str_replace_all(pattern = 'hashtag(\\w+)hashtag', replacement = '#\\1') # convert hashtags back


    # convert corpus to document term matrix
    return(payload);
}

tokenized <- clean_and_tokenize(data$content, 'english')
data.tokenized <- data %>% mutate(tokenized = tokenized)
data.tokenized %>% write_csv('./data/generated/data_with_tokenized_distinct.csv');
cat(tokenized %>% paste(collapse = '\n'), file = paste0('./data/generated/tokenized_distinct.txt'))